In [9]:
FILE = "../data/T_T100D_MARKET_US_CARRIER_ONLY_20220506_205137.zip"

import pandas as pd

from bokeh.io import output_notebook
output_notebook() 

domestic_carriers_df = pd.read_csv(
    FILE,
    compression="zip",
    usecols=range(22),
    dtype={
        "PASSENGERS": "int64",
        "FREIGHT": "int64",
        "MAIL": "int64",
        "DISTANCE": "int64",
        },
    )
domestic_carriers_df.columns = map(str.lower, domestic_carriers_df.columns)
domestic_carriers_df.head()

Loading BokehJS ...

,passengers,freight,mail,distance,unique_carrier,unique_carrier_name,carrier,carrier_name,carrier_group_new,origin_airport_id,...,origin_state_abr,origin_state_nm,dest_airport_id,dest,dest_city_name,dest_state_abr,dest_state_nm,year,quarter,month
0,0,303913,0,2547,5X,United Parcel Service,5X,United Parcel Service,3,13891,...,CA,California,14307,PVD,"Providence, RI",RI,Rhode Island,2021,4,11
1,0,47738,0,2090,5X,United Parcel Service,5X,United Parcel Service,3,13891,...,CA,California,14122,PIT,"Pittsburgh, PA",PA,Pennsylvania,2021,4,11
2,0,99406,0,1876,5X,United Parcel Service,5X,United Parcel Service,3,13891,...,CA,California,12884,LAN,"Lansing, MI",MI,Michigan,2021,4,11
3,0,1223074,0,1188,5X,United Parcel Service,5X,United Parcel Service,3,13891,...,CA,California,11298,DFW,"Dallas/Fort Worth, TX",TX,Texas,2021,4,11
4,0,475548,136603,2482,5X,United Parcel Service,5X,United Parcel Service,3,13891,...,CA,California,10529,BDL,"Hartford, CT",CT,Connecticut,2021,4,11


set up

In [10]:
# get the 25 airlines with the most passengers
biggest_carriers = domestic_carriers_df.groupby(["unique_carrier_name", "unique_carrier"])["passengers"].sum().reset_index()
biggest_carriers = biggest_carriers.sort_values(by="passengers", ascending=False)[:25]
biggest_carriers["position"] = range(1, len(biggest_carriers) + 1)  # add position column for tooltip
biggest_carriers.reset_index(drop=True, inplace=True)
biggest_carriers.head(3)

,unique_carrier_name,unique_carrier,passengers,position
0,Southwest Airlines Co.,WN,120715882,1
1,American Airlines Inc.,AA,99225144,2
2,Delta Air Lines Inc.,DL,92633601,3


In [11]:
units_dict = {
    "distance": "mi",
    "passengers": "",  # passengers don't have units
    "freight": "lbs",
    "mail": "lbs",
}


plot and daata

In [13]:
# Create dataframe with one line per month. Each line contains the sums of passengers, freight, and mail for each month for N largest carriers for that month.

import calendar

dimensions = ["passengers", "freight", "mail"]  # create list of dimensions to create sums for

df_monthly = domestic_carriers_df[domestic_carriers_df["unique_carrier"].isin(biggest_carriers["unique_carrier"])]  # only consider carriers in the biggest_carriers df
df_monthly = df_monthly.groupby(["month"])[dimensions].sum()
df_monthly["month_name"] = df_monthly.index.to_series().apply(lambda x: calendar.month_name[x])
df_monthly

,passengers,freight,mail,month_name
month,,,,
1,24348068,108929337,47355862,January
2,24490673,109774110,38546179,February
3,39345525,129078847,44790615,March
4,43803119,122441746,46691766,April
5,52699873,132180933,45443703,May
6,59987063,136600352,42192024,June
7,66541915,133733611,46621065,July
8,60458653,135988300,46963425,August
9,53867191,136175865,35474792,September


In [6]:
from bokeh.palettes import Category10
from bokeh.plotting import figure, show
from bokeh.models import NumeralTickFormatter, ColumnDataSource

plot_title = f"Development of domestic passengers, freight, and mail (top {len(biggest_carriers)} airlines by passengers)"

source = ColumnDataSource(data=df_monthly)

TOOLTIPS = "$name: $y{(0,0)}"  # simplest way: just one line, one string

largest_carriers_development_plot = figure(
    title=plot_title,
    x_range=df_monthly["month_name"],
    height=300, sizing_mode="stretch_width",
    tooltips=TOOLTIPS
)

# Configure HoverTool
largest_carriers_development_plot.hover.anchor = "center"
largest_carriers_development_plot.hover.mode = "vline"

color = 0
for measurement in dimensions:
    largest_carriers_development_plot.line(  # TBD: mke categorical (by months)?
        x="month_name",
        y=measurement,
        legend_label=f"{measurement.capitalize()} {'in' if units_dict[measurement] else ''} {units_dict[measurement]}",
        source=source,
        width=2, color=Category10[3][color], alpha=1, muted_alpha=0.2,
        name=measurement,
    )
    color += 1

largest_carriers_development_plot.yaxis.formatter = NumeralTickFormatter(format="0,0")
largest_carriers_development_plot.xaxis.axis_label = "Month"
largest_carriers_development_plot.legend.click_policy="mute"

show(largest_carriers_development_plot)